In [2]:
import numpy as np
import pandas as pd
import os

from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer, r2_score
from sklearn.base import clone

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.gaussian_process.kernels import RBF, Matern

In [3]:
from src.config import *
from src.utils import per_error, calculate_y_LOD, find_adj_score
from src.load_dataset import load_dataset
from src.load_models import select_model
from src.graph_visualization import visualization_testing_dataset

In [4]:
# Load Training Dataset
%matplotlib
X_train, X_test, y_train, y_test         = load_dataset('/Users/sangam/Desktop/Epilepsey/Code/vgramreg/ML1_ML2', standardize='min_max')
X_train_4, X_test_4, y_train_4, y_test_4 = load_dataset('/Users/sangam/Desktop/Epilepsey/Code/vgramreg/ML4', standardize='min_max')

Using matplotlib backend: MacOSX


/Users/sangam/Desktop/Epilepsey/Code/vgramreg/src/load_dataset.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.rename(columns={"PH": 'univariate, max(S)', 'signal_std':'univariate, std(S)', 'signal_mean':'univariate, mean(S)', 'peak area':'univariate, area(S)', \


######Data Distribution:#########
Training {0: 50, 16: 50, 8: 47}
Testing {0: 34, 8: 31, 16: 34}
#################################


/Users/sangam/Desktop/Epilepsey/Code/vgramreg/src/load_dataset.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.rename(columns={"PH": 'univariate, max(S)', 'signal_std':'univariate, std(S)', 'signal_mean':'univariate, mean(S)', 'peak area':'univariate, area(S)', \


######Data Distribution:#########
Training {0: 23, 8: 25, 16: 21}
Testing {0: 15, 8: 16, 16: 15}
#################################


In [5]:
# Combine
X_4, y_4 = pd.concat([X_train_4, X_test_4], axis=0), pd.concat([y_train_4, y_test_4], axis=0)
X, y     = pd.concat([X_train, X_test], axis=0), pd.concat([y_train, y_test], axis=0)

In [6]:
X

,"univariate, area(S)",peak curvature,"univariate, V_max(S)",vcenter,"univariate, max(S)","univariate, mean(S)","univariate, std(S)","univariate, max(dS/dV)","univariate, min(dS/dV)","univariate, max(dS/dV) - min(dS/dV)","univariate, V_max(dS/dV)","univariate, V_min(dS/dV)","univariate, area(dS/dV)"
0,-1.124885,-0.873945,0.087811,-1.827358,-1.129981,-1.116253,-1.145196,-1.145603,1.010617,-1.075979,-1.471324,-0.441426,-1.129649
1,1.238794,1.283919,0.635785,0.892861,1.307136,1.248902,1.294273,1.344663,-1.378694,1.366201,0.710614,0.502914,1.307415
2,-1.159712,-0.503582,-1.414581,-1.827358,-1.150254,-1.168812,-1.186024,-1.123016,1.164540,-1.148032,-2.289551,-0.846144,-1.150323
3,1.152572,1.216019,0.225712,0.212806,1.143772,1.143784,1.151376,1.077314,-1.091191,1.087391,0.437872,0.233102,1.143242
4,1.132472,1.291898,0.283775,0.552834,1.270235,1.143784,1.212617,1.558197,-1.351637,1.451257,0.301501,0.098197,1.269716
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,-0.925542,-1.335347,-1.487160,-1.827358,-0.996650,-0.932297,-0.992091,-1.078189,1.073749,-1.078397,-1.744066,-0.711238,-0.997095
95,1.106737,0.724364,0.262001,0.382820,1.104313,1.117504,1.110548,0.995611,-1.141897,1.076484,0.165130,0.098197,1.104326
96,-0.156421,-0.481995,0.715622,0.892861,-0.189464,-0.143912,-0.175532,-0.324656,0.251024,-0.285972,0.301501,0.772726,-0.189604
97,-1.145285,-1.087967,-0.264199,-1.827358,-1.132146,-1.142532,-1.155403,-1.176717,1.027252,-1.099513,-1.744066,-0.711238,-1.132082


In [7]:
models = ['Linear', 'KNN', 'SVM', 'RF', 'GP', 'Ridge', 'Lasso', 'univariate, std(S)', 'univariate, max(dS/dV)', 'univariate, area(dS/dV)', 'univariate, area(S)', 'univariate, max(S)']
metric = 'per_error'

scorer = make_scorer((r2_score if metric=='r2' else per_error), greater_is_better=(True if metric=='r2' else False))

score_dict_r2 = {'Models': [],
              'Scores': []}

score_dict_per = {'Models': [],
              'Scores': []}



training_dataset = 'ML12'
testing_dataset  = 'ML4'

X_training, y_training, X_testing, y_testing = (X, y, X_4, y_4) if training_dataset=='ML12' else (X_4, y_4, X, y)

# Calcualte y_LOD
y_LOD = calculate_y_LOD(X_testing, y_testing)
print("y_LOD", y_LOD)

for model_name in models:

    # Test for R2 score
    model    = select_model(model_name)

    model_r2  = clone(model)
    model_per = clone(model)

    model_r2.fit(X_training[models_features_r2[model_name]], y_training)                   # Fit model with R2 features on ML1_2 dataset
    y_pred_r2 = model_r2.predict(X_testing[models_features_r2[model_name]])                # Inference model with R2 features on ML4 dataset

    model_per.fit(X_training[models_features_per[model_name]], y_training)                 # Fit percent error with R2 features on ML1_2 dataset
    y_pred_per = model_per.predict(X_testing[models_features_per[model_name]])             # Inference percent error features on ML4 dataset
    
    score_dict_r2['Models'].append(model_name)
    score_dict_per['Models'].append(model_name)

    r2Score      = r2_score(y_testing, y_pred_r2)
    adj_r2_Score = find_adj_score(len(y_testing), len(models_features_r2[model_name]), r2Score)
    
    score_dict_r2['Scores'].append((r2Score, adj_r2_Score))
    score_dict_per['Scores'].append(per_error(y_testing, y_pred_per, y_LOD=y_LOD))

score_dict_r2, score_dict_per

y_LOD 0.9117010154341669


/Users/sangam/miniconda3/envs/vgramreg/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.976e+02, tolerance: 1.075e+00
  model = cd_fast.enet_coordinate_descent(


({'Models': ['Linear',
   'KNN',
   'SVM',
   'RF',
   'GP',
   'Ridge',
   'Lasso',
   'univariate, std(S)',
   'univariate, max(dS/dV)',
   'univariate, area(dS/dV)',
   'univariate, area(S)',
   'univariate, max(S)'],
  'Scores': [(0.8444378415681274, 0.8373019627409772),
   (0.8962557042567791, 0.8924831844115712),
   (0.8999401162586504, 0.8933941425559453),
   (0.8971531935638412, 0.8953166434489097),
   (0.9031429975653069, 0.9022858559508407),
   (0.850545241285834, 0.8407678271643465),
   (0.8445258026248792, 0.8373939587085892),
   (0.8711633731430644, 0.8700232260027375),
   (0.8762373044394078, 0.8751420593459512),
   (0.8730107762140706, 0.8718869777734872),
   (0.8687724004245876, 0.8676110942336548),
   (0.8730050718081576, 0.8718812228861059)]},
 {'Models': ['Linear',
   'KNN',
   'SVM',
   'RF',
   'GP',
   'Ridge',
   'Lasso',
   'univariate, std(S)',
   'univariate, max(dS/dV)',
   'univariate, area(dS/dV)',
   'univariate, area(S)',
   'univariate, max(S)'],
  'Scor

In [8]:
savedir = 'Generability_output_feature_ML12_train_ML12_test_ML4'
os.makedirs(savedir, exist_ok=True)
visualization_testing_dataset(score_dict_r2, f'{savedir}/r2_score_D_{training_dataset}_T_{testing_dataset}.png', model_name_conversion, only_one_multivariate=False, adj_score=True, legends=True)
visualization_testing_dataset(score_dict_per, f'{savedir}/per_error_score_D_{training_dataset}_T_{testing_dataset}.png', model_name_conversion, only_one_multivariate=False, r2_score=False, adj_score=False, legends=True)